# Install gdown Python package.

In [0]:
!pip install -U --no-cache-dir gdown

# Install Tensorflow-Addons.

In [0]:
!pip install tensorflow-addons

# Install Tensorflow-datasets.

In [0]:
!pip install tensorflow-datasets

# Use TensorFlow 2.x.

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

print(tf.__version__)

# Configuration parameters.

In [0]:
auto_tune = tf.data.experimental.AUTOTUNE

In [0]:
buffer_size = 1000
batch_size = 32

epochs = 60

number_of_attributes = 40
image_load_shape = (143, 143, 3)
image_shape = (128, 128, 3)

adversarial_loss_mode = 'wgan'

gradient_penalty_mode = '1-gp'
gradient_penalty_mode = 'none' # NEED TO CORRECT THIS
gradient_penalty_sample_mode = 'line'

d_gradient_penalty_weight = 10.0
d_attribute_loss_weight = 1.0
g_attribute_loss_weight = 10.0
g_reconstruction_loss_weight = 100.0

# Compute gradient penalty.

In [0]:
def _sample_line(real, fake):
    shape = [tf.shape(real)[0]] + [1] * (real.shape.ndims - 1)
    alpha = tf.random.uniform(shape=shape, minval=0, maxval=1)
    sample = real + alpha * (fake - real)
    sample.set_shape(real.shape)
    return sample


def _sample_DRAGAN(real, fake):  # fake is useless
    beta = tf.random.uniform(shape=tf.shape(real), minval=0, maxval=1)
    fake = real + 0.5 * tf.math.reduce_std(real) * beta
    sample = _sample_line(real, fake)
    return sample

In [0]:
def _norm(x):
    norm = tf.norm(tf.reshape(x, [tf.shape(x)[0], -1]), axis=1)
    return(norm)

def _one_mean_gp(grad):
    norm = _norm(grad)
    gp = tf.reduce_mean((norm - 1)**2)
    return(gp)

def _zero_mean_gp(grad):
    norm = _norm(grad)
    gp = tf.reduce_mean(norm**2)
    return(gp)

def _lipschitz_penalty(grad):
    norm = _norm(grad)
    gp = tf.reduce_mean(tf.maximum(norm - 1, 0)**2)
    return(gp)

In [0]:
def gradient_penalty(f, real, fake):
    sample_functions = {
        'line': _sample_line,
        'real': lambda real, fake: real,
        'fake': lambda real, fake: fake,
        'dragan': _sample_DRAGAN,
    }

    gradient_penalty_functions = {
        '1-gp': _one_mean_gp,
        '0-gp': _zero_mean_gp,
        'lp': _lipschitz_penalty,
    }

    if gradient_penalty_mode == 'none':
        gp = tf.constant(0, dtype=real.dtype)
    else:
        x = sample_functions[gradient_penalty_sample_mode](real, fake)
        # NEED TO CORRECT THIS
        grad = tf.gradients(f(x), x)[0] 
        gp = gradient_penalty_functions[gradient_penalty_mode](grad)

    return(gp)

# Load CelebA dataset.

In [0]:
builder = tfds.builder('celeb_a')
print(builder.info)

### Download dataset from dataset Google drive.

In [0]:
builder.download_and_prepare()

### Download dataset from Google drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp -r '/content/drive/My Drive/datasets/tensorflow_datasets' /root/.

### View dataset contents.

In [0]:
!ls -al /root/tensorflow_datasets/celeb_a/

### Create CelebA dataset splits.
* train
* validation
* test

In [0]:
celeba_datasets = builder.as_dataset()
print(celeba_datasets)

In [0]:
train_dataset = celeba_datasets['train']
val_dataset = celeba_datasets['validation']
test_dataset = celeba_datasets['test']

# Preprocess the dataset.

### Normalize the image to [-1, 1].

In [0]:
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return(image)

In [0]:
input_image = np.random.rand(image_shape[0], image_shape[1], image_shape[2])
output_image = normalize(input_image)
print('image shape',output_image.shape)

### Random crop the image.

In [0]:
def random_crop(image):
  cropped_image = tf.image.random_crop(image, size=image_shape)
  return(cropped_image)

In [0]:
input_image = np.random.rand(image_load_shape[0], image_load_shape[1], image_load_shape[2])
output_image = random_crop(input_image)
print('input image shape',input_image.shape)
print('output image shape',output_image.shape)

### Random jitter the image.

In [0]:
def random_jitter(image):  
  image = tf.image.resize(image, [image_load_shape[0], image_load_shape[1]],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)  
  image = random_crop(image)
  image = tf.image.random_flip_left_right(image)
  return(image)

In [0]:
input_image = np.random.rand(image_load_shape[0], image_load_shape[1], image_load_shape[2])
output_image = random_jitter(input_image)
print('input image shape',input_image.shape)
print('output image shape',output_image.shape)

### Preprocess train image.

In [0]:
attributes_identifiers = {'5_o_Clock_Shadow': 0, 'Arched_Eyebrows': 1, 'Attractive': 2,
          'Bags_Under_Eyes': 3, 'Bald': 4, 'Bangs': 5, 'Big_Lips': 6,
          'Big_Nose': 7, 'Black_Hair': 8, 'Blond_Hair': 9, 'Blurry': 10,
          'Brown_Hair': 11, 'Bushy_Eyebrows': 12, 'Chubby': 13,
          'Double_Chin': 14, 'Eyeglasses': 15, 'Goatee': 16,
          'Gray_Hair': 17, 'Heavy_Makeup': 18, 'High_Cheekbones': 19,
          'Male': 20, 'Mouth_Slightly_Open': 21, 'Mustache': 22,
          'Narrow_Eyes': 23, 'No_Beard': 24, 'Oval_Face': 25,
          'Pale_Skin': 26, 'Pointy_Nose': 27, 'Receding_Hairline': 28,
          'Rosy_Cheeks': 29, 'Sideburns': 30, 'Smiling': 31,
          'Straight_Hair': 32, 'Wavy_Hair': 33, 'Wearing_Earrings': 34,
          'Wearing_Hat': 35, 'Wearing_Lipstick': 36,
          'Wearing_Necklace': 37, 'Wearing_Necktie': 38, 'Young': 39}
identifiers_attributes = {v: k for k, v in attributes_identifiers.items()}

In [0]:
def compute_attributes(attributes_batch):
  number_of_samples = len(attributes_batch[identifiers_attributes[0]])
  print('number_of_samples', number_of_samples)
  attributes_array = np.zeros((number_of_samples, number_of_attributes))
  for sample_index in range(number_of_samples):
    for index in range(len(identifiers_attributes)):
      attribute = identifiers_attributes[index]
      #print(attribute, attributes_batch[attribute][sample_index].numpy()*1.0)
      attributes_array[sample_index][index] = attributes_batch[attribute][sample_index].numpy()*1.0

    #print(attributes_array[sample_index])
  return(attributes_array)

In [0]:
def compute_attributes(attributes_batch):
  attributes_array = []
  for index in range(len(identifiers_attributes)):
      attribute = identifiers_attributes[index]
      #print(attribute, attributes_batch[attribute].numpy()*1.0)
      attributes_array.append(tf.cast(attributes_batch[attribute], dtype=tf.float32))

    #print(attributes_array[sample_index])
  return(attributes_array)

In [0]:
def preprocess_train_image(sample):
  image = sample['image']
  attributes = sample['attributes']
  
  image = random_jitter(image)
  image = normalize(image)

  sample['image'] = image
  sample['attributes'] = compute_attributes(attributes)
  return(sample)

In [0]:
input_image = np.random.rand(image_load_shape[0], image_load_shape[1], image_load_shape[2])
attributes = np.random.rand(number_of_attributes)

sample = {}
sample['image'] = input_image
sample['attributes'] = attributes

output = preprocess_train_image(sample)
print('input image shape',input_image.shape)
print('output image shape',output['image'].shape)

### Preprocess test image.

In [0]:
def preprocess_test_image(sample):
  image = sample['image']
  attributes = sample['attributes']

  image = tf.image.resize(image, [image_shape[0], image_shape[1]], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR) 
  image = normalize(image)

  sample['image'] = image
  sample['attributes'] = compute_attributes(attributes)
  return(sample)

In [0]:
input_image = np.random.rand(image_load_shape[0], image_load_shape[1], image_load_shape[2])
attributes = np.random.rand(number_of_attributes)

sample = {}
sample['image'] = input_image
sample['attributes'] = attributes

output = preprocess_test_image(sample)
print('input image shape',input_image.shape)
print('output image shape',output['image'].shape)

### Preprocess dataset splits.

In [0]:
train_dataset = train_dataset.map(preprocess_train_image, num_parallel_calls=auto_tune).cache().shuffle(buffer_size).padded_batch(batch_size)

In [0]:
val_dataset = val_dataset.map(preprocess_test_image, num_parallel_calls=auto_tune).cache().shuffle(buffer_size).padded_batch(batch_size)

# Create the optimizer.

*   Adam optimizer
*   Learning rate = 0.0002
*   β1 = 0.5
*   β2 = 0.999

In [0]:
generator_optimizer = tf.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)
discriminator_optimizer = tf.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)

# Create AttGAN encoder model.

In [0]:
class UNetGenc(layers.Layer):

  def __init__(self, dimension=64, downsamplings_layers=5):
    super(UNetGenc, self).__init__()

    self._dimension = 64
    self._downsamplings_layers = 5

  def call(self, inputs):

    input_layer = inputs
    output_units = self._dimension
    
    output_layers = []
    for layer_index in range(self._downsamplings_layers):
      input_layer = layers.Conv2D(output_units, (4,4), strides=(2,2), padding='same')(input_layer)
      input_layer = layers.BatchNormalization()(input_layer)
      input_layer = layers.LeakyReLU(alpha=0.2)(input_layer)

      output_layers.append(input_layer)
      output_units = output_units * 2    
    
    return(output_layers)

In [0]:
input_image = np.random.rand(batch_size, image_shape[0], image_shape[1], image_shape[2])
encoder = UNetGenc()
output = encoder(input_image)
print('number of layers',len(output))
for layer in output:
  print('layer shape', layer.shape)

# Concatenate features and attributes.
*   Tile all elements of attributes.
*   Concat features + attributes along the channel axis.
*   features shape - (N, H, W, C_a)
*   attributes shape - (N, 1, 1, C_b) or (N, C_b)

In [0]:
def concatenate(list_of_features, list_of_attributes=[]):
  list_of_features = list(list_of_features) if isinstance(list_of_features, (list, tuple)) else [list_of_features]
  list_of_attributes = list(list_of_attributes) if isinstance(list_of_attributes, (list, tuple)) else [list_of_attributes]
  for index, attributes in enumerate(list_of_attributes):
        attributes = tf.reshape(attributes, [-1, 1, 1, attributes.shape[-1]])
        attributes = tf.tile(attributes, [1, list_of_features[0].shape[1], list_of_features[0].shape[2], 1])
        list_of_attributes[index] = attributes
  return tf.concat(list_of_features + list_of_attributes, axis=-1)

# Create AttGAN decoder model.

In [0]:
class UNetGdec(layers.Layer):

  def __init__(self, dimension=64, upsamplings_layers=5, shortcut_layers=1, inject_layers=1):
    super(UNetGdec, self).__init__()

    self._dimension = 64
    self._upsamplings_layers = 5
    self._shortcut_layers = shortcut_layers
    self._inject_layers = inject_layers

  def call(self, inputs):
    features, attributes = inputs

    #attributes = tensorflow.to_float(attributes)    
    output_units = self._dimension

    input_layer = concatenate(features[-1], attributes)
    for layer_index in range(self._upsamplings_layers - 1):
      input_layer = layers.Conv2DTranspose(output_units, (4, 4), strides=(2,2), padding='same')(input_layer)
      input_layer = layers.BatchNormalization()(input_layer)
      input_layer = layers.LeakyReLU(alpha=0.2)(input_layer)

      if (self._shortcut_layers > layer_index):
        input_layer = concatenate([input_layer, features[-2 - layer_index]])

      if (self._inject_layers > layer_index):
        input_layer = concatenate(input_layer, attributes)

      output_units = output_units * 2

    input_layer = layers.Conv2DTranspose(3, (4, 4), strides=(2,2), padding='same')(input_layer)
    input_layer = tf.keras.activations.tanh(input_layer) 

    output_layer = input_layer

    return(output_layer)  

In [0]:
input_image = np.random.rand(batch_size, image_shape[0], image_shape[1], image_shape[2])
attributes = np.random.rand(batch_size, number_of_attributes)

encoder = UNetGenc()
encoded_input = encoder(input_image)

decoder = UNetGdec()
decoded_output = decoder([encoded_input, attributes])
print(decoded_output.shape)

# Create AttGAN discriminator / classification model.

In [0]:
import tensorflow_addons as tfa

In [0]:
class Discriminator(layers.Layer):

  def __init__(self, number_of_attributes=40, dimension=64, dense_dimension=1024, downsamplings_layers=5): 
    super(Discriminator, self).__init__()  

    self._number_of_attributes = number_of_attributes
    self._dimension = dimension
    self._dense_dimension = dense_dimension
    self._downsamplings_layers = downsamplings_layers

  def call(self, inputs):

      input_layer = inputs  
      output_units = self._dimension  

      for layer_index in range(self._downsamplings_layers): 
        input_layer = layers.Conv2D(output_units, (4,4), strides=(2,2), padding='same')(input_layer)         
        input_layer = tfa.layers.InstanceNormalization()(input_layer)
        input_layer = layers.LeakyReLU(alpha=0.2)(input_layer)

        output_units = output_units * 2

      input_layer = layers.Flatten()(input_layer)

      discriminator_output = layers.Dense(self._dense_dimension)(input_layer) 
      discriminator_output = tfa.layers.InstanceNormalization()(discriminator_output)     
      discriminator_output = layers.LeakyReLU(alpha=0.2)(discriminator_output)
      discriminator_output = layers.Dense(1)(discriminator_output)
      
      attribute_output = layers.Dense(self._dense_dimension)(input_layer)  
      attribute_output = tfa.layers.InstanceNormalization()(attribute_output)         
      attribute_output = layers.LeakyReLU(alpha=0.2)(attribute_output)
      attribute_output = layers.Dense(self._number_of_attributes, activation='sigmoid')(attribute_output)

      return([discriminator_output, attribute_output])

In [0]:
input_image = np.random.rand(batch_size, image_shape[0], image_shape[1], image_shape[2])

discriminator = Discriminator(number_of_attributes)
discriminator_prediction, attribute_prediction = discriminator(input_image)

print('discriminator prediction shape', discriminator_prediction.shape)
print('attribute prediction shape', attribute_prediction.shape)

# Create adversarial loss functions.
*   Generator loss function
*   Discriminator loss function

## WGAN loss functions.
*   Generator loss function
*   Discriminator loss function

In [0]:
def wgan_loss_functions():
    def discriminator_loss_function(real_logit, fake_logit):
        real_loss = - tf.reduce_mean(real_logit)
        fake_loss = tf.reduce_mean(fake_logit)
        return(real_loss, fake_loss)

    def generator_loss_function(fake_logit):
        fake_loss = - tf.reduce_mean(fake_logit)
        return(fake_loss)

    return(discriminator_loss_function, generator_loss_function)

In [0]:
def adversarial_loss_functions(adversarial_loss_mode):
  if(adversarial_loss_mode == 'wgan'):
    return(wgan_loss_functions())
  else:
    return(wgan_loss_functions())

# Create different models and loss functions.
* Encoder model
* Decoder model
* Discriminator model
* Discriminator loss function
* Generator loss function

In [0]:
encoder = UNetGenc()
decoder = UNetGdec()
discriminator = Discriminator(number_of_attributes)

discriminator_loss_function, generator_loss_function = adversarial_loss_functions(adversarial_loss_mode)

# Create composite generator model.

In [0]:
def create_composite_generator(image_shape, number_of_attributes):

  xa = layers.Input(shape=image_shape, name='input_image')
  a = layers.Input(shape=(number_of_attributes), name='attributes')

  b = tf.random.shuffle(a)

  a_ = a * 2 - 1
  b_ = b * 2 - 1

  # Generate
  z = encoder(xa)
  xa_ = decoder([z, a_])
  xb_ = decoder([z, b_])

  # Discriminate
  xb__logit_gan, xb__logit_att = discriminator(xb_)

  xb__loss_gan = generator_loss_function(xb__logit_gan)
  xb__loss_att = tf.keras.losses.categorical_crossentropy(b, xb__logit_att)
  xa__loss_rec = tf.keras.losses.mae(xa, xa_)

  loss = (xb__loss_gan + 
            xb__loss_att * g_attribute_loss_weight +
            xa__loss_rec * g_reconstruction_loss_weight)

  composite_model = tf.keras.models.Model([xa, a], [loss])  

  return(composite_model)

In [0]:
composite_generator = create_composite_generator(image_shape, number_of_attributes)

# Create composite discriminator model.

In [0]:
def create_composite_discriminator(image_shape, number_of_attributes):

  xa = layers.Input(shape=image_shape)
  a = layers.Input(shape=(number_of_attributes))

  b = tf.random.shuffle(a)

  a_ = a * 2 - 1
  b_ = b * 2 - 1

  # Generate
  z = encoder(xa)  
  xb_ = decoder([z, b_])

  # Discriminate
  xa_logit_gan, xa_logit_att = discriminator(xa)
  xb__logit_gan, xb__logit_att = discriminator(xb_)

  # Discriminator losses
  xa_loss_gan, xb__loss_gan = discriminator_loss_function(xa_logit_gan, xb__logit_gan)
  gp = gradient_penalty(lambda x: discriminator(x)[0], xa, xb_)
  xa_loss_att = tf.losses.categorical_crossentropy(a, xa_logit_att)

  loss = (xa_loss_gan + xb__loss_gan +
            gp * d_gradient_penalty_weight +
            xa_loss_att * d_attribute_loss_weight)
  
  composite_model = tf.keras.models.Model([xa, a], [loss])  

  return(composite_model)

In [0]:
composite_discriminator = create_composite_discriminator(image_shape, number_of_attributes)

# Train the model.

In [0]:
import matplotlib.pyplot as plt

sample_image = next(iter(train_dataset))

plt.subplot(121)
plt.title('sample image')
plt.imshow(sample_image['image'][0] * 0.5 + 0.5)
print(sample_image['attributes'][0])

In [0]:
for dataset_batch in train_dataset:
  attributes = dataset_batch['attributes']
  images = dataset_batch['image']
  size = len(images)
  print(size)
  print(attributes)

In [0]:
def train(train_dataset, val_dataset, epochs=60):
  for epoch in range(epochs):
    for dataset_batch in train_dataset:

      attributes = dataset_batch['attributes']
      images = dataset_batch['image']

      with tf.GradientTape(persistent=True) as tape:
        composite_discriminator_loss = composite_discriminator([images, attributes], training=True)
        composite_generator_loss = composite_generator([images, attributes], training=True)

      composite_discriminator_gradients = tape.gradient(composite_discriminator_loss, composite_discriminator.trainable_variables)
      composite_generator_gradients = tape.gradient(composite_generator_loss, composite_generator.trainable_variables)

      discriminator_optimizer.apply_gradients(zip(composite_discriminator_gradients, composite_discriminator.trainable_variables))
      generator_optimizer.apply_gradients(zip(composite_generator_gradients, composite_generator.trainable_variables))
  

In [0]:
train(train_dataset, val_dataset, epochs)